# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [34]:
#Your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

%matplotlib inline



music_df = pd.read_csv('homepage_actions.csv')
music_df.shape

(8188, 4)

In [35]:
# Lets check the statistics of the dfs id column

music_df.describe()

,id
count,8188.000000
mean,564699.749878
std,219085.845672
min,182988.000000
25%,373637.500000
50%,566840.500000
75%,758078.000000
max,937217.000000


In [36]:
#Lets us also check the summary on the df
music_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [37]:
# Here we will view the clicks on the action column
# There are 1860 clicks both appearing on experiment and control groups
music_df.loc[music_df['action'] == 'click']

,timestamp,id,group,action
8,2016-09-24 20:58:01.948663,349125,experiment,click
12,2016-09-24 21:06:27.553057,601714,experiment,click
15,2016-09-24 21:30:02.739756,487634,experiment,click
17,2016-09-24 23:01:12.108316,468601,experiment,click
21,2016-09-25 00:01:47.933853,555973,experiment,click
...,...,...,...,...
8162,2017-01-17 23:20:35.483601,451198,control,click
8164,2017-01-17 23:47:58.209653,252195,control,click
8167,2017-01-18 00:56:24.554729,344770,experiment,click
8180,2017-01-18 08:54:56.879682,615849,experiment,click


In [38]:
music_df['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [39]:
# We look at anyone who clicked but did not view and viewed without clicking
view = music_df.loc[music_df['action'] == 'view']
click = music_df.loc[music_df['action'] == 'click']

print('views without clicking: {}'.format( len(set(view['id'].to_list()) - set(click['id'].to_list())) ))
print('clicks without viewing: {}'.format( len(set(click['id'].to_list()) - set(view['id'].to_list())) ))

views without clicking: 4468
clicks without viewing: 0


In [40]:
# Here we look for an overlap between users in control and experiment groups
experiment = music_df.loc[music_df['group'] == 'experiment']
control = music_df.loc[music_df['group'] == 'control']

print('users in both experiment and control: {}'\
      .format( len(set(experiment['id'].to_list()) & set(control['id'].to_list())) ))

users in both experiment and control: 0


In [41]:
# Findings from investigating the music_df 

"""
1860 people viewed then clicked, 
No anomalies for people who clicked without viewing, viewed or clicked more than once)
"""

'\n1860 people viewed then clicked, \nNo anomalies for people who clicked without viewing, viewed or clicked more than once)\n'

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [53]:
#Your code here
# I will conduct a one-tailed t-test
# Formulate a hypothesis and choose our alpha
"""
H0 : Experimental homepage and control group have identical click rates
H1 : Experimental homepage has higher click rates than control group
alpha = 0.05
"""

'\nH0 : Experimental homepage and control group have identical click rates\nH1 : Experimental homepage has higher click rates than control group\nalpha = 0.05\n'

In [50]:
music_df['val'] = 1
control = music_df.loc[music_df['group'] == 'control'].pivot(index = 'id',columns='action',values='val').fillna(0)
experiment = music_df.loc[music_df['group'] == 'experiment'].pivot(index = 'id',columns='action',values='val').fillna(0)

stats.ttest_ind(control.click, experiment.click)

Ttest_indResult(statistic=-2.6195696844542202, pvalue=0.008825098914958293)

In [52]:
# Findings

"""
Our results show that p-value of approximately 0.0088 is less than our alpha of 0.005.

We therefore reject the null hypothesis, there is evidence to show that the 
experimental homepage is more effective than that of the control group


"""

'\nOur results show that p-value of approximately 0.0088 is less than our alpha of 0.005.\n\nWe therefore reject the null hypothesis, there is evidence to show that the \nexperimental homepage is more effective than that of the control group\n\n\n'

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [54]:
#Your code here

n = experiment.view.sum()
p = control.click.mean()
experiment_expected_clicks = n*p 
experiment_expected_clicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [55]:
#Your code here
stdev = np.sqrt(n*p*(1-p))
z = (experiment.click.sum() - experiment_expected_clicks) / stdev
z

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [56]:
#Your code here
stats.norm.sf(z)


0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Yes, though the p-value is slighly lower, both are less than alpha of 0.05.
This means that we reject the null hypotheis and can conclude that experimental page appears to be a most effective design**


## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.